### Load Data and Test Likelihood Function

In [ ]:
import pandas as pd
import numpy as np
from skyfield.data import mpc
from skyfield.api import load, N, W, wgs84
from misc import epoch2jd, chi2fun, ln_like

# Load Sun Earth ephemeris
eph = load('MPC/de421.bsp')
sun, earth = eph['sun'], eph['earth']
# Site
obs_site = earth + wgs84.latlon(35.904613 * N, 79.046761 * W)
# Asteroid
target = '2012FN62'
# Load orbital elements
with load.open(f'MPC/MPCORB.{target}.DAT') as f:
    minor_planets = mpc.load_mpcorb_dataframe(f)
row = minor_planets.loc[0].copy() # Create an explicit copy

# template orbital elements (Panda Series)
elms = row.copy()

# array of truth element
truths = np.array([row.semimajor_axis_au,
                   row.eccentricity,
                   row.inclination_degrees,
                   row.longitude_of_ascending_node_degrees,
                   row.argument_of_perihelion_degrees,
                   row.mean_anomaly_degrees])
# MPC epoch_packed to JD
truths_T = epoch2jd(row.epoch_packed)

# load csv file
datafile = '2012FN62_s150_n3'
cat = pd.read_csv(f'sim_data/{datafile}.csv')
jds = np.array(cat.JD)
ras = np.array(cat.RA_obs)
decs = np.array(cat.Dec_obs)
perr = np.array(cat.perr)

# parameter names and boundaries
params = ['$a$','$e$','$i$','$\Omega$', '$\omega$', '$MA$']
bounds = np.array([[2.0,   0,  0,   0,  0,    0],
                   [3.5, 0.9, 30, 360, 360, 360]])

# test Likelihood and Chi^2 functions
# Set print options to display 2 decimal places
np.set_printoptions(formatter={'float': '{:.2f}'.format})
print(f"(a,e,i,Omega,omega,MA) @ Epoch: {truths} @ {truths_T}")
print(ln_like(truths,bounds,elms,jds,ras,decs,perr))
print(chi2fun(truths,elms,jds,ras,decs,perr))

### AMOEBA

Use a downhill simplex method with simulated annealing (`amoeba_sa`) 
to search for the best-fit orbital elements in the large 6-dimensional 
parameter space. 

In [ ]:
""" 
Computing Time on M2 Macbook Air: 
    2m for 10 iterations of nmax = 1000 steps
"""
from amoeba import amoeba_sa

def func(p): return chi2fun(p,elms,jds,ras,decs,perr)

# Parameters for simulated annealing
niter = 10
cooling = 1.0 / niter
temp0 = len(jds)
scl0 = (bounds[1,:]-bounds[0,:])/3
p0 = (bounds[1,:]+bounds[0,:])/2

# Set print options to display 2 decimal places
np.set_printoptions(formatter={'float': '{:.2f}'.format})

# truth chisq and initial guess chisq
print(f"(a,e,i,Omega,omega,MA): {truths}")
print(f"Truth chi-squared: {func(truths)}")
print(f"(a,e,i,Omega,omega,MA): {p0}")
print(f"Initial chi-squared: {func(p0)}")

# Run simulated annealing
for i in range(niter):
    temp = temp0 * (1.0 - cooling * i)
    scl = scl0 * (1.0 - cooling * i / 2.0)
    best_params, result = amoeba_sa(1e-5, function_name=func, 
                                    p0=p0, scale=scl, 
                                    upper=bounds[1,:], lower=bounds[0,:],
                                    temperature=temp, nmax=1000)
    p0 = best_params.copy()  # Use the best parameters as the starting point for the next iteration
    # show progress
    print(f"(a,e,i,Omega,omega,MA): {best_params}")
    print(f"Minimum chi-squared: {np.min(result['chi2'])}")

### Compare Predicted Positions with True Positions on a Longer Baseline

In [ ]:
import matplotlib.pyplot as plt
from misc import predict_pos

# extend the JD range by 600 days
jdextra = np.linspace(np.min(jds),np.min(jds)+600)
# predict positions based on true elements
theta = truths.copy()
ra_true, dec_true, _ = predict_pos(theta, elms, jdextra)
# predict positions based on best-fit elements
theta = best_params.copy()
ra_pred, dec_pred, _ = predict_pos(theta, elms, jdextra)
# plot the RA and Dec residuals
plt.plot(jdextra-np.min(jds),ra_true._degrees-ra_pred._degrees,'-',
         jdextra-np.min(jds),dec_true._degrees-dec_pred._degrees,'--')
# show observed dates
plt.plot(jds-np.min(jds),[0,0,0],'bo')
plt.axhline(0,linestyle=':');

# print predicted postions at the observing times
print(jds,ras, decs)
theta = best_params.copy()
ra_pred, dec_pred, _ = predict_pos(theta, elms, jds)
print(ra_pred._degrees,dec_pred._degrees)

### Run MCMC sampler

In [ ]:
""" MCMC-sampling with multiprocessing """
""" 100%|██████████| 1000/1000 [00:52<00:00, 19.06it/s] """
""" MCMC-sampling with single processing """
""" 100%|██████████| 1000/1000 [04:09<00:00,  4.01it/s] """

import multiprocessing, emcee, os
import numpy as np

# prep for multithreading
multiprocessing.set_start_method("fork")   # fork: copy a Python process from an existing process.
os.environ["OMP_NUM_THREADS"] = "1"        # disable automatic parallelization in NumPy

# need 8500 steps for chains to converge: nsteps/max(tau) > 50
nsteps = 10000 
nrepeat = 1
converge_check = True

emcfile = f'results/{datafile}.h5'
if os.path.exists(emcfile): os.remove(emcfile)

""" initial starting positions of the walkers """
cnter = best_params
print(cnter)
ncpu = 8
ndim = len(bounds[0,:])
nwalkers = ncpu*int(np.ceil(2.5*ndim/ncpu)) 
scale = (bounds[1,:]-bounds[0,:])/50
pos = cnter + scale*(np.random.normal(size=(nwalkers, ndim)))
# clip initial positions outside of bounds
for i in range(ndim):
    pos[:,i] = np.clip(pos[:,i], bounds[0,i], bounds[1,i]) 

for irepeat in range(1,nrepeat+1):
    print(f'\nThis is {irepeat=} out of {nrepeat=}')
    # set up backend to save MCMC samples
    backend = emcee.backends.HDFBackend(emcfile)
    if not os.path.exists(emcfile): 
        print(f'Reset backend because {emcfile} not present')
        backend.reset(nwalkers, ndim)
        print(f'Start MCMC from initial random positions')
        instate = pos
        chainexist = False
    else: 
        print(f'Start MCMC from where left off the last time in {emcfile}')
        instate = None
        chainexist = True

    # set up sampler 
    with multiprocessing.Pool() as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, 
                ln_like, args=(bounds,elms,jds,ras,decs,perr), 
                pool=pool, backend=backend)
        # check if chain is already long enough
        if chainexist and converge_check:
            chainshape = sampler.get_chain().shape
            tau = sampler.get_autocorr_time(quiet=True)
            flat_samples = sampler.get_chain(flat=True) 
            log_probs = sampler.get_log_prob(flat=True)
            map_idx = np.argmax(log_probs)  # Index of the MAP sample
            map_params = flat_samples[map_idx]   # Parameters at MAP
            print(f"MAP: {log_probs[map_idx]}, parameters: {map_params}")
            if chainshape[0] > 50*np.max(tau): 
                print("[Break]: Chain length already exceeds 50x Auto-correlation Time")
                break 
        # start walking
        sampler.run_mcmc(instate, nsteps, progress=True)


### Corner Plots to Show Covariances amongst Orbital Elements

In [ ]:
""" make a corner plot over full bounded range """
import emcee, corner
import matplotlib.pyplot as plt

sampler = emcee.backends.HDFBackend(emcfile)
nstep, nwalker, ndim = sampler.get_chain().shape
# use trimmed, thinned, flattened sample for corner plots
tau = sampler.get_autocorr_time(quiet=True)
burnin = int(2 * np.max(tau))
thin = int(0.5 * np.min(tau))
flat_samples = sampler.get_chain(discard=burnin, flat=True, thin=thin)
mids = np.array([np.percentile(flat_samples[:,i],50) for i in range(ndim)])
_ = corner.corner(flat_samples, labels=params, truths=truths,
        title_quantiles=[0.16,0.50,0.84], quantiles=[0.50],
        show_titles=True, plot_contours=False, plot_density=False, bins=50,
        range=list(zip(bounds[0],bounds[1])))
plt.savefig(f'{emcfile.replace(".h5","")}_{nstep}.png', bbox_inches='tight')

In [ ]:
_ = corner.corner(flat_samples, labels=params, truths=truths,
        title_quantiles=[0.16,0.50,0.84], quantiles=[0.50],
        show_titles=True, plot_contours=False, plot_density=False, bins=50)
plt.savefig(f'{emcfile.replace(".h5","")}_{nstep}_zoom.png', bbox_inches='tight')